In [ ]:
# 🧱 基础导入
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# 数据变换
transform = transforms.Compose([
    transforms.Resize(32),   # VGG expects 32x32
    transforms.ToTensor()
])

# 加载数据
print("loading data")
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=False)

class VGG11(nn.Module):
    def __init__(self):
        super(VGG11, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device", device)
model = VGG11().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, loader):
    model.train()
    correct, total, running_loss = 0, 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, pred = torch.max(outputs, 1)
        total += y.size(0)
        correct += (pred == y).sum().item()
    return running_loss / len(loader), correct / total

def evaluate(model, loader):
    model.eval()
    correct, total, loss = 0, 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss += criterion(outputs, y).item()
            _, pred = torch.max(outputs, 1)
            total += y.size(0)
            correct += (pred == y).sum().item()
    return loss / len(loader), correct / total


train_losses, test_losses = [], []
train_accs, test_accs = [], []

for epoch in range(10):
    train_loss, train_acc = train(model, trainloader)
    test_loss, test_acc = evaluate(model, testloader)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Test Acc={test_acc:.4f}")

# 绘图
epochs = range(1, 11)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_accs, label="Train Acc")
plt.plot(epochs, test_accs, label="Test Acc")
plt.title("Accuracy vs Epoch")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_losses, label="Train Loss")
plt.plot(epochs, test_losses, label="Test Loss")
plt.title("Loss vs Epoch")
plt.legend()
plt.show()


loading data
using device cpu


In [ ]:
def test_with_transform(transform):
    testset_flip = torchvision.datasets.MNIST(root='./data', train=False, download=True,
                                               transform=transforms.Compose([
                                                   transforms.Resize(32),
                                                   transform,
                                                   transforms.ToTensor()
                                               ]))
    loader = DataLoader(testset_flip, batch_size=128, shuffle=False)
    _, acc = evaluate(model, loader)
    return acc

h_flip_acc = test_with_transform(transforms.RandomHorizontalFlip(p=1))
v_flip_acc = test_with_transform(transforms.RandomVerticalFlip(p=1))

print(f"Horizontal Flip Accuracy: {h_flip_acc:.4f}")
print(f"Vertical Flip Accuracy: {v_flip_acc:.4f}")

In [ ]:
def noise_test(std):
    noise_transform = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x + std * torch.randn_like(x))
    ])
    noisy_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True,
                                               transform=noise_transform)
    noisy_loader = DataLoader(noisy_testset, batch_size=128, shuffle=False)
    _, acc = evaluate(model, noisy_loader)
    return acc

for var in [0.01, 0.1, 1]:
    std = var ** 0.5
    acc = noise_test(std)
    print(f"Gaussian noise (var={var}): Accuracy = {acc:.4f}")

In [ ]:
augment_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
aug_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=augment_transform)
aug_trainloader = DataLoader(aug_trainset, batch_size=128, shuffle=True)

# 重新训练模型
model_aug = VGG11().to(device)
optimizer = optim.Adam(model_aug.parameters(), lr=0.001)

for epoch in range(10):
    train_loss, train_acc = train(model_aug, aug_trainloader)
    test_loss, test_acc = evaluate(model_aug, testloader)
    print(f"[Augmented] Epoch {epoch+1}: Train Acc={train_acc:.4f}, Test Acc={test_acc:.4f}")